# **Workflow for Sentinel-1 TOPS Coregistrastion Stack**

> This workflow is based on the Sentinel-1 TOPS Coregistrastion Processing Stack in ISCE2, which is designed for Dolphin processing.
 
**Minimum Requirements:**
- [install ISCE2](https://github.com/InSARProcessing/isce2/blob/develop/isce%E5%AE%89%E8%A3%85%E7%AC%94%E8%AE%B0.md)
- [Sentinel-1 TOPS data](https://search.asf.alaska.edu/#/)
- Local SRTM GL1 DEM tiles -> You don't have to unzip them.

**Steps:** </br>
- Download TOPS precise orbit files </br>
- Download auxliary calibration files </br>
- Prepare SRTM GL1 mosaic DEM </br>
- Stack TOPS data </br>
- Generate configs </br>
- Run ISCE2 stack </br>

🛠️Before the stack starts, we need to configure the environment variables and the paths for processing.

In [4]:
import os
import re
import glob

# 1. Input ISCE repository location
isce_install = "/home/wang_xuanxuan/tools/isce2"

os.environ["PATH"] = os.path.join(isce_install, "install", "bin") + ":" + os.environ["PATH"]
# Check if the PYTHONPATH exists, if not, initialize it as an empty string
pythonpath = os.environ.get("PYTHONPATH", "")
os.environ["PYTHONPATH"] = os.path.join(isce_install, "install", "packages") + ":" + pythonpath

# 2. Input workspace and meta data location
WORKSPACE = "/mnt/e/InSAR/Stuttgart/ISCE"          # The workspace prepared for ISCE2 processing
SLC_LOCATION = "/mnt/e/InSAR/Stuttgart/RAW"         # The location of Sentinel-1 TOPS data
LOCAL_DEM_DIR = "/mnt/e/DEM/SRTMGL1/"               # The location which you stored all the local SRTM GL1 DEM tiles

# Then automatically set up the orbit and auxliary calibration files directory
ORBIT_DIR = os.path.join(WORKSPACE, "ORB")
AUX_DIR = os.path.join(WORKSPACE, "AUX")
os.makedirs(WORKSPACE, exist_ok=True)
os.makedirs(ORBIT_DIR, exist_ok=True)
os.makedirs(AUX_DIR, exist_ok=True)

👀 This is a double-check step to confirm the SLC files for processing, provided by the variable `SLC_LOCATION`

In [6]:
# Locate all SAFE files in the directory
safe_files = glob.glob(os.path.join(SLC_LOCATION, 'S1*_IW_SLC*'))
if not safe_files:
    print(f"No SAFE files found in directory: {SLC_LOCATION}")

print(f"Found {len(safe_files)} SAFE files")

Found 31 SAFE files


Now the setting is done. Let's start the workflow ! ☺️

<b> 1. Download precise orbit files </b>

In [ ]:
import requests
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

def _get_orbit_list() -> list[str]:
    """
    从 ASF（Alaska Satellite Facility）服务器获取 Sentinel-1 卫星轨道文件列表。
    
    该函数从 ASF 的轨道文件存储库获取所有可用的精密轨道文件（.EOF 格式）列表。
    这些轨道文件包含卫星的精确位置信息，用于 InSAR 数据处理中的几何校正和配准。
    
    Returns:
        list[str]: 轨道文件名列表（.EOF 格式）。如果获取失败，返回空列表。
    """
    url_root = "https://s1qc.asf.alaska.edu/aux_poeorb"

    try:
        session = _create_session()
        response = session.get(url_root, timeout=30)
        response.raise_for_status()

        # print(response.text)
        # Extract all .EOF files from the HTML
        orbit_files = re.findall(r'href="([^"]*\.EOF)"', response.text)
        print(f"Found {len(orbit_files)} orbit files.")
        return orbit_files

    except Exception as e:
        print(f"Failed to get orbit list: {e}")
        return []

def _create_session() -> requests.Session:
    """
    创建一个带有重试策略的 requests 会话。
    
    配置了自动重试机制，用于处理网络请求中的常见错误（如服务器错误、超时等），
    提高下载轨道文件的可靠性。
    
    Returns:
        requests.Session: 配置好重试策略的会话对象。
    """
    session = requests.Session()

    # Setup retry strategy
    retry_strategy = Retry(
        total=3,
        backoff_factor=1,
        status_forcelist=[429, 500, 502, 503, 504],
    )

    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("http://", adapter)
    session.mount("https://", adapter)

    return session

# Download Precise Orbit
from datetime import datetime, timedelta
orbit_list = _get_orbit_list()
if not orbit_list:
    print("Failed to get orbit file list from ASF server")

In [ ]:
def _download_orbit_file_asf(orbit_file: str, output_path: os.PathLike) -> bool:
    url_root = "https://s1qc.asf.alaska.edu/aux_poeorb"
    file_url = f"{url_root}/{orbit_file}"

    try:
        session = _create_session()
        response = session.get(file_url, stream=True, timeout=60)
        response.raise_for_status()

        # Get file size
        total_size = int(response.headers.get('content-length', 0))
        downloaded_size = 0

        with open(output_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file.write(chunk)
                    downloaded_size += len(chunk)

                    # Show progress
                    if total_size > 0:
                        progress = (downloaded_size / total_size) * 100
                        print(f"\r  Progress: {progress:.1f}%", end='', flush=True)

        print()  # New line after progress
        return True

    except Exception as e:
        print(f"\n  Download failed: {e}")
        # Clean up partially downloaded file
        if output_path.exists():
            output_path.unlink()
        return False

print(f"Start downloading orbit files from ASF...")
# For each SAFE file, download the corresponding orbit file
for safe_file in safe_files:
    print("* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */\n")
    print(f"Processing: {os.path.basename(safe_file)}")
    
    # Extract mission and timestamp from SAFE 
    basename = os.path.basename(safe_file)

    # Remove .SAFE or .zip extension
    if basename.endswith('.SAFE'):
        basename = basename[:-5]
    elif basename.endswith('.zip'):
        basename = basename[:-4]

    # Split by underscores
    parts = basename.split('_')

    if len(parts) < 6:
        print(f"  Invalid filename format: {basename}")
        continue

    # Mission identifier (S1A or S1B)
    mission = parts[0]

    # Timestamps are typically at positions 5 and 6
    # Format: YYYYMMDDTHHMMSS
    start_time_str = parts[5]
    stop_time_str = parts[6]

    # Parse timestamps
    try:
        start_time = datetime.strptime(start_time_str, '%Y%m%dT%H%M%S')
        stop_time = datetime.strptime(stop_time_str, '%Y%m%dT%H%M%S')
    except ValueError:
        print(f"  Could not parse timestamps from: {basename}")
        continue

    if not mission or not start_time:
        print("  Could not parse SAFE filename, skipping")
        continue

    # Calculate date range for orbit search (one day before and after)
    date1 = (start_time - timedelta(days=1)).strftime('%Y%m%d')
    date2 = (start_time + timedelta(days=1)).strftime('%Y%m%d')

    # Find matching orbit file
    mission_short = mission[-1]
    # Search for matching orbit file
    for orbit_file in orbit_list:
    # Check if file matches mission and contains both dates
        if (f"S1{mission_short}" in orbit_file and
            date1 in orbit_file and date2 in orbit_file):
            # Check if orbit file already exists in orbit directory
            orbit_path = os.path.join(ORBIT_DIR, orbit_file)
            if os.path.exists(orbit_path):
                print(f"  Orbit file already exists: {orbit_file}")
                continue
            # Download orbit file
            success = _download_orbit_file_asf(orbit_file, orbit_path)

            if success:
                print(f"  Successfully downloaded: {orbit_file}")
            else:
                print(f"  Failed to download orbit file for: {os.path.basename(safe_file)}")
    
print("* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */\n")
print(f"Finished downloading orbit files.")

<b> 2. Download AUX files </b>

In [ ]:
# Download aux
import subprocess
print(f"Start downloading auxiliary files from SAR-MPC...")
print("* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */\n")
for slc_file in safe_files:
    filename = os.path.basename(slc_file)
    wget_cmd = ['wget', 'https://sar-mpc.eu/download/ca97845e-1314-4817-91d8-f39afbeff74d/',
                '-O', filename]
    aux_file = os.path.join(AUX_DIR, filename)
    if os.path.exists(aux_file):
        print(f"  Auxiliary file already exists: {aux_file}")
        print("* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */\n")
        continue
    subprocess.run(wget_cmd, cwd=AUX_DIR, check=True)
    print(f"Downloaded auxiliary file for {filename} to {AUX_DIR}. \n")
    print("* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */\n")

print(f"Finished downloading auxiliary files.")


<b> 3. Download DEM </b>

In [ ]:
# Download DEM
import zipfile
# First, customize a function to detect the lat/lon region of the SLCs
lon_list = []
lat_list = []
polygons = []
print(f"Start preparing DEM ...")
print("* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */\n")
for safe_file in safe_files:
    if safe_file.endswith(".SAFE"):
        try:
            # For unzipped SAFE files, find the KML file in the preview directory
            kml_file = glob.glob(os.path.join(safe_file, "preview", f"map-overlay.kml"))
            if kml_file:
                with open(kml_file[0], 'r', encoding='utf-8') as kml:  # Fixed variable name and encoding
                    content = kml.read()
        except Exception as e:
            raise IOError(f"Error extracting KML from SAFE file {safe_file}: {e}")
        
    elif safe_file.endswith(".zip"):
        # For zipped SAFE files, directly read the KML file from inside the ZIP
        try:
            with zipfile.ZipFile(safe_file, 'r') as zip_ref:
                # Search for the map-overlay.kml file in the preview directory
                kml_entries = [entry for entry in zip_ref.namelist() if "preview/map-overlay.kml" in entry]
                if kml_entries:
                    # Read the KML content directly from the ZIP file
                    with zip_ref.open(kml_entries[0]) as kml_file:
                        content = kml_file.read().decode('utf-8', errors='ignore')
        except Exception as e:
            raise IOError(f"Error processing ZIP file {safe_file}: {e}")

    coord_pattern = r'<coordinates[^>]*>(.*?)</coordinates>'
    coord_matches = re.findall(coord_pattern, content, re.DOTALL | re.IGNORECASE)

    for coord_block in coord_matches:
        points = []
        coord_text = coord_block.strip()
        coord_text = re.sub(r'\s+', ' ', coord_text)
        coord_triplets = coord_text.split()

        for coord_str in coord_triplets:
            if ',' in coord_str:
                parts = coord_str.split(',')
                if len(parts) >= 2:
                    try:
                        lon = float(parts[0])
                        lat = float(parts[1])
                        if -180 <= lon <= 180 and -90 <= lat <= 90:
                            points.append((lon, lat))
                    except ValueError:
                        pass

    if points:
        # Remove duplicate closing point if exists
        if len(points) > 1 and points[0] == points[-1]:
            points = points[:-1]
        polygons.append(points)
    
if polygons and len(polygons) > 0:
    lons = [pt[0] for pt in polygons[0]]
    lats = [pt[1] for pt in polygons[0]]
    lon_list.extend(lon for lon in lons)
    lat_list.extend(lat for lat in lats)

# Find the common lat/lon region of the SLCs
DEM_REGION = (int(min(lat_list)), int(max(lat_list)) + 1, int(min(lon_list)), int(max(lon_list)) + 1)

os.chdir(LOCAL_DEM_DIR)
dem_cmd = f"dem.py -a stitch -b {DEM_REGION[0]} {DEM_REGION[1]} {DEM_REGION[2]} {DEM_REGION[3]} -r -s 1 -c -k -l -d {LOCAL_DEM_DIR}"
print(f"Executing: {dem_cmd}")
! {dem_cmd}
dem_output = os.path.join(LOCAL_DEM_DIR, f"demLat_N{DEM_REGION[0]:02d}_N{DEM_REGION[1]:02d}_Lon_E{DEM_REGION[2]:03d}_E{DEM_REGION[3]:03d}.dem.wgs84")
print("* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */\n")
print(f"Finished downloading and mosaicing DEM - {dem_output}")


Start preparing DEM ...
* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */

Executing: dem.py -a stitch -b 47 51 7 12 -r -s 1 -c -l -d /mnt/e/DEM/SRTMGL1/
Using default ISCE Path: /home/wang_xuanxuan/tools/isce2/install/packages/isce
Could not create a stitched DEM. Some tiles are missing
N50E007.SRTMGL1.hgt.zip = failed
N50E008.SRTMGL1.hgt.zip = failed
N50E009.SRTMGL1.hgt.zip = failed
N50E010.SRTMGL1.hgt.zip = failed
N50E011.SRTMGL1.hgt.zip = failed
N49E007.SRTMGL1.hgt.zip = failed
N49E008.SRTMGL1.hgt.zip = failed
N49E009.SRTMGL1.hgt.zip = failed
N49E010.SRTMGL1.hgt.zip = failed
N49E011.SRTMGL1.hgt.zip = failed
N48E007.SRTMGL1.hgt.zip = failed
N48E008.SRTMGL1.hgt.zip = failed
N48E009.SRTMGL1.hgt.zip = failed
N48E010.SRTMGL1.hgt.zip = failed
N48E011.SRTMGL1.hgt.zip = failed
N47E007.SRTMGL1.hgt.zip = failed
N47E008.SRTMGL1.hgt.zip = failed
N47E009.SRTMGL1.hgt.zip = failed
N47E010.SRTMGL1.hgt.zip = failed
N47E011.SRTMGL1.hgt.zip = failed
* - 

<b> 4. Generate configs </b>

In [ ]:
os.environ["PYTHONPATH"] += os.pathsep + os.path.join(isce_install, "src", "isce2", "contrib", "stack")
os.environ["PATH"] += os.pathsep + os.path.join(isce_install, "src", "isce2", "contrib", "stack", "topsStack")

In [ ]:
if os.path.exists(os.path.join(WORKSPACE, "run_files")):
    for file in os.listdir(os.path.join(WORKSPACE, "run_files")):
        os.remove(os.path.join(WORKSPACE, "run_files", file))
    os.rmdir(os.path.join(WORKSPACE, "run_files"))
# Run stackSentinel.py
!stackSentinel.py -s {SLC_LOCATION} -o {ORBIT_DIR} -a {AUX_DIR} -d {dem_output} -w {WORKSPACE} -c 3 -r 10 -z 2 -m 20250109 -C NESD -W slc -useGPU

<b> 5. Run steps (all in one) </b>

In [ ]:
os.environ["PYTHONPATH"] += os.pathsep + os.path.join(isce_install, "src", "isce2", "contrib", "stack")
os.environ["PATH"] += os.pathsep + os.path.join(isce_install, "src", "isce2", "contrib", "stack", "topsStack")

os.chdir(WORKSPACE)
run_files_dir = os.path.join(WORKSPACE, "run_files")
for file in os.listdir(run_files_dir):
    cmd = f"bash ./run_files/{file} 2>&1 | tee -a ./run_all.log"
    !{cmd}

<b> 6. (Alternative) Run steps (one by one) </b></br>
This is the alternative way to run the steps one by one, in order to understand the workflow.

In [ ]:
os.chdir(WORKSPACE)
!bash ./run_files/run_01_unpack_topo_reference 2>&1 | tee ./run_01_unpack_topo_reference.log
print("* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */\n")
print(f"Finished unpacking topo reference.")


In [ ]:
os.chdir(WORKSPACE)
!bash ./run_files/run_02_unpack_secondary_slc 2>&1 | tee ./run_02_unpack_secondary_slc.log
print("* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */\n")
print(f"Finished unpacking secondary SLCs.")


In [ ]:
os.chdir(WORKSPACE)
!bash ./run_files/run_03_average_baseline 2>&1 | tee ./run_03_average_baseline.log
print("* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */\n")
print(f"Finished averaging baseline.")



In [ ]:
os.chdir(WORKSPACE)
!bash ./run_files/run_04_extract_burst_overlaps 2>&1 | tee ./run_04_extract_burst_overlaps.log
print("* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */\n")
print(f"Finished extracting burst overlaps.")


In [ ]:
os.chdir(WORKSPACE)
!bash ./run_files/run_05_overlap_geo2rdr 2>&1 | tee ./run_05_overlap_geo2rdr.log
print("* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */\n")
print(f"Finished geo2rdr for overlap bursts.")


In [ ]:
os.chdir(WORKSPACE)
!bash ./run_files/run_06_overlap_resample 2>&1 | tee ./run_06_overlap_resample.log
print("* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */\n")
print(f"Finished resampling overlap bursts.")


In [ ]:
os.chdir(WORKSPACE)
!bash ./run_files/run_07_pairs_misreg 2>&1 | tee ./run_07_pairs_misreg.log
print("* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */\n")
print(f"Finished misregistration for pairs.")

In [ ]:
os.chdir(WORKSPACE)
!bash ./run_files/run_08_timeseries_misreg 2>&1 | tee ./run_08_timeseries_misreg.log
print("* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */\n")
print(f"Finished misregistration for timeseries.")

In [ ]:
os.chdir(WORKSPACE)
!bash ./run_files/run_09_fullBurst_geo2rdr 2>&1 | tee ./run_09_fullBurst_geo2rdr.log
print("* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */\n")
print(f"Finished geo2rdr for full bursts.")

In [ ]:
os.chdir(WORKSPACE)
!bash ./run_files/run_10_fullBurst_resample 2>&1 | tee ./run_10_fullBurst_resample.log
print("* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */\n")
print(f"Finished resampling full bursts.")

In [ ]:
os.chdir(WORKSPACE)
!bash ./run_files/run_11_extract_stack_valid_region 2>&1 | tee ./run_11_extract_stack_valid_region.log
print("* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */\n")
print(f"Finished extracting stack valid region.")

In [ ]:
os.chdir(WORKSPACE)
!bash ./run_files/run_12_merge_reference_secondary_slc 2>&1 | tee ./run_12_merge_reference_secondary_slc.log
print("* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */\n")
print(f"Finished merging reference and secondary SLC.")

In [ ]:
os.chdir(WORKSPACE)
!bash ./run_files/run_13_grid_baseline 2>&1 | tee ./run_13_grid_baseline.log
print("* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */\n")
print(f"Finished gridding baseline.")

That's all for the SLC Coregistration process in ISCE2. Now you can check the results in the `WORKSPACE` directory. Enjoy it ：）））

Now, for **Dolphin** use, an automatic process is designed.


In [ ]:
os.chdir(WORKSPACE)
# In case, jax-cuda is not configured right for the Dolphin use, 
!dolphin config \
    --cslc merged/SLC/*/*.slc.full \
    --worker-settings.no-gpu-enabled \
    --worker-settings.threads-per-worker 4 \
    --unwrap-options.unwrap-method ICU

In [ ]:
os.chdir(WORKSPACE)
!dolphin run dolphin_config.yaml